# Прогнозирование цены финансовых инструментов. Построение модели нейронный сети 

На первом этапе производится загрузка необходимых для работы стандартных библиотек

In [1]:
import pandas as pd
import numpy as np

import sklearn.model_selection
import sklearn.metrics

import tensorflow as tf
import keras.layers
import keras.models
import keras.callbacks

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout

2022-07-05 04:35:39.806671: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-05 04:35:39.806697: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Импорт данных 

Вторым этапом проводится импорт данных и объединение таблиц в одну. Основная объединенная таблица, содержащая в себе все имеющиеся данные, сохранена в переменной df. 

In [2]:
df = pd.concat(map(pd.read_csv, ['03.06.2022.csv', '06.06.2022.csv', '07.06.2022.csv', '15.06.2022.csv', '19.05.2022.csv', '20.05.2022.csv', '21.06.2022.csv', '23.05.2022.csv', '23.06.2022.csv', '24.05.2022.csv', '25.05.2022.csv']))


## Рассмотрение имеющихся данных 

До построения модели необходимо рессмотреть детали имеющегося наора данных. В первую очередь на экран выведен объем набора данных. Имеющийся набор данных состоит из 1 544 865 строк и 7 слолбцов (включая столбец с нумерацией): 

In [3]:
df.shape

(1544865, 7)

Далее выведен фрагмент объединенной таблицы и шапка таблицы для визуального обзора:  

In [4]:
df

,Unnamed: 0,trade_date,trade_time,shares_bid,bid,ask,shares_ask
0,0,2022-06-03,15:59:56,8.0,5.82,5.83,1600.0
1,1,2022-06-03,15:59:56,100.0,5.82,5.83,1404.0
2,2,2022-06-03,15:59:56,900.0,5.82,5.83,100.0
3,3,2022-06-03,15:59:56,100.0,5.82,5.83,32.0
4,4,2022-06-03,15:59:56,700.0,5.81,5.83,4.0
...,...,...,...,...,...,...,...
147877,147877,2022-05-25,09:30:01,100.0,5.41,6.27,100.0
147878,147878,2022-05-25,09:30:01,100.0,5.38,6.28,100.0
147879,147879,2022-05-25,09:30:01,100.0,5.34,6.31,100.0
147880,147880,2022-05-25,09:30:01,100.0,5.26,6.35,100.0


Ниже представлен более детальный обзор набора данных, где указан тап данных в каждым столбце, количество ненулевых переменных, объем памяти, занимаемый данными.  Из обзора видно, что в объединенной таблице имеются пропуски данных и имеются нечисловые значения. Для дальнейшего построения, обучения и тестирования модели глубокой нейронной сети это может быть критично, так как в основном модели работают только с числовыми значениями и без пропусков данных. Чистка данных будет производиться в следующем подразделе после изучения объединенной таблицы. 

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1544865 entries, 0 to 147881
Data columns (total 7 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   Unnamed: 0  1544865 non-null  int64  
 1   trade_date  1544865 non-null  object 
 2   trade_time  1544865 non-null  object 
 3   shares_bid  1471300 non-null  float64
 4   bid         1471300 non-null  float64
 5   ask         1471300 non-null  float64
 6   shares_ask  1471300 non-null  float64
dtypes: float64(4), int64(1), object(2)
memory usage: 94.3+ MB


Сразу будет логично посмотреть количество пропущенных значений в наборе данных: 

In [8]:
df.isnull().sum()

Unnamed: 0        0
trade_date        0
trade_time        0
shares_bid    73565
bid           73565
ask           73565
shares_ask    73565
dtype: int64

Далее проведен небольшой анализ по имеющимся взаимосвязям между числовыми переменными в колонках набора данных (корреляция) с переменной "Ask", котороая является целевой переменной для предсказаний цены финансового инструмента. Из анализа видно, что наиболее существенная связь у переменной "Ask" наблюдается с переменной "bid": 

In [5]:
df[df.columns[:]].corr()['ask'][:].sort_values(ascending=False)

ask           1.000000
bid           0.992826
Unnamed: 0    0.111023
shares_ask   -0.002451
shares_bid   -0.014422
Name: ask, dtype: float64

В завершение подраздела по изучению набора данных приведена таблица с основными статистическими показателями переменных: минимиум (min), макимум (max), среднее (mean), стандартное отлонение(std), общее количсетво (count)  и  показатели с учетом процентного соотношения: 

In [7]:
df.describe().round(1)

,Unnamed: 0,shares_bid,bid,ask,shares_ask
count,1544865.0,1471300.0,1471300.0,1471300.0,1471300.0
mean,71353.5,191.4,5.9,5.9,174.0
std,42509.8,312.6,0.3,0.3,316.0
min,0.0,1.0,4.7,5.0,1.0
25%,35110.0,100.0,5.8,5.8,100.0
50%,70221.0,100.0,5.9,5.9,100.0
75%,105331.0,200.0,6.2,6.2,100.0
max,168251.0,8900.0,6.5,6.8,20700.0


## Чистка данных 

В подразделе по чистке данных в первую очередь избавимся от ненужных столбцов, таких как нумерция, чтобы модель не искала взаимосвязи с данным столбцом, который не несет в себе практической пользы и смысла в части предсказания цены финансового инструмента. 

In [9]:
df = df.drop(['Unnamed: 0'], axis=1)

Далее избавляемся от пропущенных значений: 

In [11]:
df = df.dropna()
print("df_cleaned:", df.shape)

df_cleaned: (1471300, 7)


Как было отмечено ранее, некоторые столбцы нуждаются в предобрзовании формата данны на числовой. Такие столбцы как "trade_date" и "trade_time" важны с точки зрения поиска закономерностей, но при этом для того, чтобы модель приняла их во внимание, нужно их формат с тип "object" изменить на более подходящий:  

In [14]:
df['trade_date'] = pd.to_datetime(df.trade_date,format='%Y-%m-%d')

In [15]:
df['trade_date'] = df['trade_date'].apply(pd.Timestamp.timestamp)

In [16]:
df['trade_time'] = pd.to_datetime(df.trade_time,format='%H:%M:%S')

In [17]:
df['trade_time'] = df['trade_time'].apply(pd.Timestamp.toordinal)

## Разделение наборов данных на тестовые и обучающие 

Для дальнейшего обучения модели требуется разделение наборов данных на тестовые и обучающие. Тестовый набор будет сохранен в переменной df_test, а тренировочный в переменной - df_train: 

In [19]:
df_train, df_test = sklearn.model_selection.train_test_split(df)
print("df_train:", df_train.shape)
print("df_test:", df_test.shape)

df_train: (1103475, 6)
df_test: (367825, 6)


Также необходимо разделить полученные тренирвоочный и тестовый сеты на X и Y, чтобы передать эту структуру модели. Для этого оставляем в переменной X весь набор данных кроме целевой переменной "ask", а в переменной Y, наоборот, оставляем только переменную "ask" и в дельнейшем передадим ее модели как ту, которую надо предсказать, основываясь на данные X.  

В результате выведен полученный объем строк и столбцов в разделенных наборах данных, где видно, что датасет X содержит в себе 5 столбцов, а датасет Y всего один: 

In [20]:
x_train = df_train.drop(["ask"], axis=1)
y_train = df_train["ask"]
print("x_train:", x_train.shape)
print("y_train:", y_train.shape)

x_test = df_test.drop(["ask"], axis=1)
y_test = df_test["ask"]
print("x_test:", x_test.shape)
print("y_test:", y_test.shape)

x_train: (1103475, 5)
y_train: (1103475,)
x_test: (367825, 5)
y_test: (367825,)


Чтобы в результате сформированного проекта по предстказанию цены финансового инструмента была возможность проверить на реальных данных корректность предстказаний и сравнить их с уже меющющимися, сформируем набор тестовый данных для предсказаний: 

In [21]:
x_test_predict = df_test.drop(["ask"], axis=1)

## Нормализация значений

Нормализация — это подход, который применяется во время подготовки данных для изменения значений числовых столбцов в наборе данных для использования общей шкалы, когда объекты в данных имеют разные диапазоны. Учитывая, что на этапе обучения моделей глубокого обучения входные данные каждого слоя постоянно меняются, обучение таких моделей является сложной задачей и данные должны быть подготовлены заранее. Нормалиция значений проводится по шкале от 0 до 1. При этом нормализация проводится как для тренировочного, так и для тестового датасета, так как они оба будут участвовать в манипуляциях, производимых с модельью.       

In [22]:
df_train = df_train.values
df_test = df_test.values

In [23]:
min_max = MinMaxScaler(feature_range=(0, 1))
x_train = min_max.fit_transform(x_train)

In [24]:
min_max = MinMaxScaler(feature_range=(0, 1))
x_test = min_max.fit_transform(x_test)

После нормализации значений требуется трансформация формы наборов данных в трехмерное измерение для построения модели: 

In [27]:
x_train = x_train.reshape(x_train.shape[0], 5, 1)
print ("x_train shape", x_train.shape)

x_test = x_test.reshape(x_test.shape[0], 5, 1)
print ("x_test shape", x_test.shape)

x_train shape (1103475, 5, 1)
x_test shape (367825, 5, 1)


## Построение модели глубокой нейронной сети

Для задачи, поставленной в рамках данного проекта, была выбрана модель глубокой нейронной сети LSTM (Long Short Term Memory). Сети с долговременной кратковременной памятью (LSTM) представляют собой тип рекуррентной нейронной сети, способной изучать зависимость порядка в задачах прогнозирования последовательности. Особенностью LSTM является то, что данная модель может выборочно "забывать" и "запоминать" последовательности и взаимосвязи. Основной стратегией LSTM является возможность "оглянуться назад" на отчеты о за предыдущие периоды. LSTM вводит единицы памяти, называемые состояниями ячеек, такие сконструированные ячейки можно рассматривать как дифференцируемую память.

In [28]:

model = Sequential()
# 1-й слой с регуляризацией Dropout
# * unit = добавить 100 нейронов — размерность выходного пространства
# * return_sequences = True, чтобы сложить слои LSTM, чтобы следующий слой LSTM имел ввод трехмерной последовательности
# * input_shape => Форма обучающего набора данных
model.add(LSTM(units=100, return_sequences=True, input_shape=(x_train.shape[1],1)))
# 20% слоев будут удалены
model.add(Dropout(0.2))
# 2-й слой LSTM
# * unit = добавить 50 нейронов — размерность выходного пространства
# * return_sequences = True, чтобы сложить слои LSTM, чтобы следующий слой LSTM имел ввод трехмерной последовательности
model.add(LSTM(units=50, return_sequences=True))
# 20% слоев будут удалены
model.add(Dropout(0.2))
# 3-й слой LSTM
# * unit = добавить 50 нейронов — размерность выходного пространства
# * return_sequences = True, чтобы сложить слои LSTM, чтобы следующий слой LSTM имел ввод трехмерной последовательности
model.add(LSTM(units=50, return_sequences=True))
# 50% слоев будут удалены
model.add(Dropout(0.5))
# 4-й слой LSTM
# * unit = добавить 50 нейронов — размерность выходного пространства
model.add(LSTM(units=50))
# 50% слоев будут удалены
model.add(Dropout(0.5))
# Слой, указывающий вывод в одного unit
model.add(Dense(units=1))
# Вывод информации о сформированной модели 
model.summary()


2022-07-05 04:35:54.915654: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-05 04:35:54.915685: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-05 04:35:54.915709: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Ubuntu-1804-bionic-64-minimal): /proc/driver/nvidia/version does not exist
2022-07-05 04:35:54.915928: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 5, 100)            40800     
_________________________________________________________________
dropout (Dropout)            (None, 5, 100)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 5, 50)             30200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 50)             0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 5, 50)             20200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 5, 50)             0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                2

В итоге получена модель глубокого обучения нейронной сети с 4-мя слоями и 111 451 параметрами. Данее необходимо определить метрики модели для оценки ее качества и предказательной способности. 

Учитывая, что изначальной целью проекта является предсказание цены финансового инструмента, то данную задачу стоит рассматривать как задачу регрессии для алгоритмов машинного обучения. Стандартные метрики для оценки модели регерсии это: 
- средний квадрат отклонений (mean_squared_error)
- потери валидации (val_loss)

Оба значения должны быть минимизированы, так как являются сумарным значением отклонений и ошибок, полученных при обучении и тестировании модели. 

In [29]:
defined_metrics = [
    tf.keras.metrics.MeanSquaredError(name='MSE')
]

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, mode='min', verbose=1)

model.compile(optimizer='adam', loss='mean_squared_error', metrics=defined_metrics)

## Обучение и тестирование модели глубокой нейронной сети

Финальный шаг в создании модели - это ее обучение на тренировочном наборе данныз и тестирование на тестовом наборе данных. В результате проведения данной манипуляции будет видна производительность модели и ее качество предсказания, а именно значения получившихся метрик качества (mean_squared_error, val_loss):  

In [30]:
history = model.fit(x_train, 
                    y_train, 
                    epochs=15, 
                    batch_size=10, 
                    validation_data=(x_test, y_test),
                    shuffle=True,
                    callbacks=[callback])

Epoch 1/15


2022-07-05 04:35:55.585746: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


110348/110348 [==============================] - 1480s 13ms/step - loss: 0.0647 - MSE: 0.0647 - val_loss: 0.0019 - val_MSE: 0.0019
Epoch 2/15
110348/110348 [==============================] - 1475s 13ms/step - loss: 0.0034 - MSE: 0.0034 - val_loss: 0.0025 - val_MSE: 0.0025
Epoch 3/15
110348/110348 [==============================] - 1478s 13ms/step - loss: 0.0031 - MSE: 0.0031 - val_loss: 0.0017 - val_MSE: 0.0017
Epoch 4/15
110348/110348 [==============================] - 1476s 13ms/step - loss: 0.0030 - MSE: 0.0030 - val_loss: 0.0018 - val_MSE: 0.0018
Epoch 5/15
110348/110348 [==============================] - 1478s 13ms/step - loss: 0.0029 - MSE: 0.0029 - val_loss: 0.0021 - val_MSE: 0.0021
Epoch 6/15
110348/110348 [==============================] - 1481s 13ms/step - loss: 0.0029 - MSE: 0.0029 - val_loss: 0.0019 - val_MSE: 0.0019
Epoch 00006: early stopping


Финальный результат тестирования модели показал довольно низкое отклонение и ошибку, равную 0,0019. Данная величина ошибки вполне приемлема и модель при таких низких ошибках должна показывать достаточно высокую предсказательную способность цены финансового инструмента. 

## Предсказания модели

В финале проекта построим таблицу с предсказаниями цены "ask"на основе построенной модели:

In [115]:
predictions = model.predict(x_test)

Теперь для того, чтобы оценить реальный результат и увидеть способность модели более нагладяно сформируем таблицу, в которой покажем реальные данные из оригинального набора данных (тестовую часть, на которой модель не обучалась) и сопоставим с предсказаниями (столбец "predictions"):

In [116]:
predictions = pd.DataFrame(predictions, columns=["predictions"], index=x_test_predict.index)

In [118]:
combination = pd.concat([x_test_predict[["ask"]], predictions], axis=1)
combination.sample(10)

,ask,predictions
60039,6.10,6.125231
2785,6.20,6.190581
33453,5.23,5.208787
128640,6.18,6.139150
156788,6.32,6.314339
151128,5.83,5.818227
38528,5.85,5.825417
72515,5.69,5.696556
66677,6.10,6.109018
96017,5.84,5.827636


Для наглядности можно добавить столбец "разница между реальным значением и предсказанием", чтобы более наглядно было видно, насколько точно предстказывает модель:  

In [123]:
combination ['разница между реальным значением и предсказанием'] = combination ["ask"] - combination ["predictions"]
combination.sample (10)

,ask,predictions,разница между реальным значением и предсказанием
64285,5.95,5.948954,0.001046
16147,5.30,5.219943,0.080057
121771,5.39,5.342470,0.047530
28646,6.15,6.156361,-0.006361
109779,6.04,5.995697,0.044303
40204,5.91,5.897795,0.012205
3161,6.20,6.183565,0.016435
83195,5.84,5.822773,0.017227
38433,5.91,5.932046,-0.022046
96957,5.95,5.950216,-0.000216


## Выводы

В рамках данного проекта были загружены и проанализированы данные финансовых показателей за некоторый промежуток времени. Финальной целью проекта являлось предсказание цены финансового инструмента, расположенной в столбце "ask". Для реализации данной цели была построена модель нейронной сети глубокого обучения и достигнут результат с высокой предсказательной способностью и средней квадратической ошибкой менее 0,002. 

Далее для имплементации данной модели в продуктив необходимо добавить переменную "predictions", сформированной на основании модели "model". Переменную "predictions" будет отображать предсказываемую цену в новом наборе данных, содержащих ту же структуру, что была отображена в настоящем проекте. Работа по имплементации модели в продуктив проводится дата-инженером. 